In [13]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

In [14]:
cd ..

/home/kgh6784/private/project/instance_segmentation


In [15]:
cd mmdetection

/home/kgh6784/private/project/instance_segmentation/mmdetection


In [16]:
import torch
import mmcv
import terminaltables
import pycocotools
from pycocotools.coco import COCO
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import build_dataset
from mmdet.datasets.coco import CocoDataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector
from mmdet.apis import show_result_pyplot
from mmcv.runner import load_checkpoint
from mmdet.apis import set_random_seed
import mmcv

In [17]:
import pylab
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
dataset_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset'

In [7]:
# from requests import get 

# def download(url, file_name):
#     with open(file_name, "wb") as file:   
#         response = get(url)               
#         file.write(response.content)


# url = "https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_x101_32x4d_fpn_mstrain-poly_3x_coco/mask_rcnn_x101_32x4d_fpn_mstrain-poly_3x_coco_20210524_201410-abcd7859.pth"
# download(url,"mask_rcnn_x101_32x4d_fpn_mstrain-poly_3x_coco_20210524_201410-abcd7859.pth")

In [ ]:
cd shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/

In [23]:
cfg.train_pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
 {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'Normalize',
  'mean': [123.675, 116.28, 103.53],
  'std': [58.395, 57.12, 57.375],
  'to_rgb': True},
 {'type': 'Pad', 'size_divisor': 32},
 {'type': 'DefaultFormatBundle'},
 {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]

In [22]:
cfg.train_pipeline['type'] = 'hi'

TypeError: list indices must be integers or slices, not str

In [19]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d'),
        groups=32,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(


In [18]:

# 사용할 모데 초기 py 설정
config = '/home/kgh6784/private/project/instance_segmentation/mmdetection/configs/mask_rcnn/mask_rcnn_x101_32x4d_fpn_1x_coco.py'

# config 불러오기
cfg = mmcv.Config.fromfile(config)

# 데이터셋 지정
cfg.dataset_type = 'CocoDataset'

# data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
data_root = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/'

# class 설정
cfg.classes = ('Normal',)

# train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

cfg.data.samples_per_gpu=8
cfg.data.workers_per_gpu=2

cfg.data.train.data_root = data_root 
cfg.data.train.ann_file = 'modify.json' 
#cfg.data.train.ann_file = 'label(polygon)_train.json' 
cfg.data.train.img_prefix = 'train/'
cfg.data.train.classes = cfg.classes



cfg.data.val.data_root = data_root
#cfg.data.val.ann_file = 'label(polygon)_train.json'
cfg.data.train.ann_file = 'modify.json' 
cfg.data.val.img_prefix = 'train/'
cfg.data.val.classes = cfg.classes

cfg.data.test.data_root = data_root
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'test/'
cfg.data.test.classes = cfg.classes

# 클래스 수 지정
cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.mask_head.num_classes = 1

# 사전 훈련 모델 지정
cfg.load_from = '/home/kgh6784/private/project/instance_segmentation/mmdetection/checkpoint/mask_rcnn_x101_32x4d_fpn_1x_coco_20200205-478d0b67.pth'

# 가중치 저장 위치
cfg.work_dir = '/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/'

# learning rate
cfg.optimizer.lr = 0.02 / 8

# 로그 출력 시기 설정
cfg.log_config.interval = 1 
cfg.lr_config.policy = 'step'

# 평가 지표로 설정
cfg.evaluation.metric = ['bbox', 'segm']

# 평가 구간 설정
cfg.evaluation.interval = 10

# 체크포인트 구간 설정
cfg.checkpoint_config.interval = 10

# epoch 설정
cfg.runner = dict(type='EpochBasedRunner', max_epochs=5)
cfg.runner.max_epochs = 10
# 결과 재현을 위한 시드값 설정
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'

In [ ]:
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES

# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
img = mmcv.imread('/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/test/0087.jpg')

model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

In [12]:
# 해당하는 폴더에 예측 json 저장 하는 코드입니다.

!python /home/kgh6784/private/project/instance_segmentation/mmdetection/test.py /home/kgh6784/private/project/instance_segmentation/mmdetection/configs/custom/lg_mask_r101.py /home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/latest.pth --format-only --eval-options "jsonfile_prefix=/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/"

/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 350/350, 2.1 task/s, elapsed: 167s, ETA:     0s